In [30]:
# Import necessary dependencies
import os
import sys
import pathlib
import json
from typing import Dict

import numpy as np
import pandas as pd
from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.metrics.pairwise import cosine_similarity, cosine_distances

import warnings
warnings.filterwarnings('ignore')

from cos_sim import *

# Define relative path
current_dir = os.getcwd()
project_dir = os.path.dirname(current_dir)

USERDATA = pathlib.Path(os.path.join(project_dir, 'userInfo'))
userdata_filepaths = list(USERDATA.glob('*.json'))